## Import Packages

In [16]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from collections import deque
from pylab import *
%matplotlib inline
import os
os.listdir("test_images/")
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [17]:
# TUNING TOOL 
def apply_brightness_contrast(input_img, brightness = 0, contrast = 0):
    
    """a function to add contrast and brightness: this is only used to as a visual help to fine tune
        some of the pipeline parameters.
    """  

    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            highlight = 255
        else:
            shadow = 0
            highlight = 255 + brightness
        alpha_b = (highlight - shadow)/255
        gamma_b = shadow

        buf = cv2.addWeighted(input_img, alpha_b, input_img, 0, gamma_b)
    else:
        buf = input_img.copy()
        #buf=np.copy(input_img)
    if contrast != 0:
        f = 131*(contrast + 127)/(127*(131-contrast))
        alpha_c = f
        gamma_c = 127*(1-f)

        buf = cv2.addWeighted(buf, alpha_c, buf, 0, gamma_c)

    return buf

In [18]:

# COLOR FILTERING FUNCTIONS

def HSL_color_picker(H,S,L):
    """
    # HLS values have to beconverted in [0..255] values for cv2 usage 
    # H value can be arbitrary, [0 ... 360](degrees on the color wheel ) (OpenCV: [0 ... 180]!!!)
    # L  [1 ... 100] % on the luminosity axis (OpenCV: [0 ... 255]): low~shade(i.e black),high~color
    # S [1 ... 100] % on the satuation axis (OpenCV: [0 ... 255])] low~faded(i.e white) high~normal lighting"""
    
    if H not in range(0,361):
        print("Hue should be in range(0,360) ")

    if S not in range(0,255):
        print("Saturation tu should be in range(0,256) ")
    if L not in range(0,255):
        print("Luminosity should be in range(0,256) ")
    return(np.array([np.round( H/2), np.round(255*L/100 ), np.round(255*S/100)]))

    
    #return(np.array([np.round( H/2), np.round(255*S/100 ), np.round(255*L/100)]))
    
    #white_upper = np.array([np.round(360 / 2), np.round(1.00 * 255), np.round(0.30 * 255)])


def filter_color_mask(image_RGB,lower_filter = np.array([0,0,0]), higher_filter = np.array([0,0,0])):#create a
    # "image" : format is RGB, 
    # filters are Hue, Luminosity, Saturation
    image_HLS=cv2.cvtColor(image_RGB, cv2.COLOR_RGB2HLS) #conversion to HLS
    mask = cv2.inRange(image_HLS, lower_filter, higher_filter)
    return(mask)


In [19]:

class AveragedBuffer():
    
    """stores values in a buffer(FIFO),limited length, 
        and return the mean for the non zero values 
    """   
    
    def __init__(self,length):
        
        self.length=length
        self.deck=deque([],maxlen=self.length)
        
    def append(self,element):
        self.deck.append(element)
    
    def content(self):
        #for testing and tuning purposes
        return(list(self.deck))
    
    def MEAN(self):
        v = np.array(self.deck, dtype=np.float32)
    
        if sum(v)==0:
            return(0)
        else:
            v[v == 0] = np.nan #to exclude the O values from the average calculation 
            return (np.nanmean(v))
        

In [20]:

            
def line_to_XY(line):
    
    """draws a straight line between a couple of points"""
    
    (x1,y1,x2,y2)=line[0]                   
    X=[x1,x2]
    Y=[y1,y2]
    return(X,Y)
    
def slope_line(line):
    
    """calculates the slope and the intercept passing through a couple of points"""
    
    (X,Y)=line_to_XY(line)
    return(polyfit(X,Y,1))    


   
def STD_filtering(img,lines,DEVIATION_FACTOR=4,color=[0,255,0],thickness=5):
    
    """filters out oulliers from a list of point couples
    returns the slope and intercept of the best fit line for the remaining pony couples"""
    
    if lines is None:
        return(0,0)
    else:
        N_lines=lines
        # calculate the slope for each segments on this side:
        N_SLOPES=list(map(lambda line:(slope_line(line))[0],N_lines))
        
        # calculates the mean for all slopes encountered on this side:
        MEAN_SLOPE=np.mean(N_SLOPES)
    
        # calculates the corresponding standard deviation: 
        STD_DEV_m=np.std(N_SLOPES) 
        
        # finds out if each segment is within a span centered around the mean slope(FYI: "<=" make it work when only one segment(STD=0)
        IN_STD_DEV_m=lambda line:MEAN_SLOPE-(DEVIATION_FACTOR*1*(STD_DEV_m))<=(slope_line(line))[0]<=MEAN_SLOPE+(DEVIATION_FACTOR*1*(STD_DEV_m))
    
        # filters out each segment whose slope is not within the mean centered span:
        LINES_IN_STD_DEV_m=list(filter(IN_STD_DEV_m,N_lines))#excluding the lines not within a DEVIATION_FACTOR span centered around MEAN_SLOPE
        N_SLOPES_in_std=list(map(lambda line:(slope_line(line))[0],LINES_IN_STD_DEV_m))
        

        # calculates the intercept for each remaining segments on this side (reminder: slope_line(line)[1] is the intercept)
        N_INTERCEPTS=list(map(lambda line:slope_line(line)[1],LINES_IN_STD_DEV_m))
    
        # calculates the mean for all corresponding intercepts remaining on this side:
        MEAN_INTERCEPT=np.mean(N_INTERCEPTS)
    
        # calculates the corresponding intercept standard deviation:
        STD_DEV_b=np.std(N_INTERCEPTS) #(mx+b)
    
        # finds out if each segment is within a span centered around the mean intercept:
        IN_STD_DEV_b=lambda line:MEAN_INTERCEPT-(DEVIATION_FACTOR*2*(STD_DEV_b))<=(slope_line(line))[1]<=MEAN_INTERCEPT+(DEVIATION_FACTOR*2*(STD_DEV_b))

        # filters out each segment whose intercept is not within the mean centered span:
        LINES_IN_STD_DEV_b=list(filter(IN_STD_DEV_b,LINES_IN_STD_DEV_m))
    
        # we'll use linear regression to find out the best fitting lines passing through the remaining segments 
    
        if len(LINES_IN_STD_DEV_b)>0: # only create lines when sub-lines have been detected on this side
            X_N=np.array([])
            Y_N=np.array([])
    
            for line in LINES_IN_STD_DEV_b:
                X_N=np.append(X_N,line_to_XY(line)[0])
                Y_N=np.append(Y_N,line_to_XY(line)[1])
   
            (m,b)=polyfit(X_N,Y_N,1) #linear regression
    
            return(m,b)
     
 
        else:
            return(0,0) #no relevant average line was detected on this side of this image  
    
    

In [21]:
def draw_side_Top2Bottom(img,avg_slope,avg_intercept,m,b,color=[255,0,0]):
    
    """average the last 10 consecutive slopes and intercept"""  
    
    if m!=0 and b!=0:
        if avg_slope.MEAN()!=0:
            if abs((m-avg_slope.MEAN())/avg_slope.MEAN())<0.2:
            #only non erratic slope are added to the average
                avg_slope.append(m)
                avg_intercept.append(b)
        else:
            avg_slope.append(m)
            avg_intercept.append(b)
               
            
        m=avg_slope.MEAN()
        b=avg_intercept.MEAN()
        
    if m!=0 and b!=0:
        NL_bottom=(int (polyval([m**-1, -b/m],YBOT)), int(YBOT))
        NL_top=(int (polyval([m**-1, -b/m],YTOP)), int(YTOP))         
        cv2.line(img,NL_bottom,NL_top,color=[255,0,255],thickness=10) 
    else :
        print("no line detected ")
      


In [22]:

def split_view(img, lines, thickness=5):
    
    """splits a list of point couples (right or left)
    send these lists for further processing: STD filtering, slope and intercept filtering
    and lane drawing"""
         
    if lines is None:
        
        # no line was detected on this image
        draw_side_Top2Bottom(img,right_average_SLOPE,right_average_INTERCEPT,m=0,b=0)
        draw_side_Top2Bottom(img,left_average_SLOPE,left_average_INTERCEPT,m=0,b=0)
                
    else:
        
        #right side:
        R_lines=list(filter(lambda line:slope_line(line)[0]>0, lines))
        
        (m,b)=STD_filtering(img,R_lines,DEVIATION_FACTOR=1,color=[255,0,0],thickness=5)
        draw_side_Top2Bottom(img,right_average_SLOPE,right_average_INTERCEPT,m,b)
      
        #left side:
        L_lines=list(filter(lambda line:slope_line(line)[0]<0, lines))
        (m,b)=STD_filtering(img,L_lines,DEVIATION_FACTOR=2,color=[255,0,0],thickness=5)
        draw_side_Top2Bottom(img,left_average_SLOPE,left_average_INTERCEPT,m,b)


In [23]:

class AveragedBuffer():
    
    """stores values in a buffer(FIFO),limited length, 
        and return the mean for the non zero values 
    """   
    
    def __init__(self,length):
        
        self.length=length
        self.deck=deque([],maxlen=self.length)
        
    def append(self,element):
        self.deck.append(element)
    
    def content(self):
        #for testing and tuning purposes
        return(list(self.deck))
    
    def MEAN(self):
        v = np.array(self.deck, dtype=np.float32)
    
        if sum(v)==0:
            return(0)
        else:
            v[v == 0] = np.nan #to exclude the O values from the average calculation 
            return (np.nanmean(v))
        

In [24]:

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), min_line_len, max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    split_view(line_img,lines)

    return(line_img)


def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)
    


In [25]:
def process_image(image):
 
    if image.shape[:1]!=(540, 960):
        image=cv2.resize(image,(960,540))
        
    image_sharper=apply_brightness_contrast(cv2.medianBlur(image,5),0,100)

    white_mask=filter_color_mask(image,lower_white,higher_white)
    yellow_mask = filter_color_mask(image,lower_yellow,higher_yellow)

    mask = white_mask+yellow_mask
    mask=mask.astype(np.uint8)
    
    image_color_mask=cv2.bitwise_and(image_sharper, image_sharper, mask = mask)

    #image_color_mask=apply_masks_OR(image_sharper,mask_1=white_mask,mask_2=yellow_mask) 

    Gray_Ex=grayscale(image_color_mask) # tranform it in a 2D(8bits) gray image


    Blurred_Gray=gaussian_blur(Gray_Ex, 5) # convolves averaging square size 5 filter
    
    Edges=cv2.Canny(Blurred_Gray, 50, 100)
    
    Masked_Edges=region_of_interest(Edges,VERTICES) #  region_of_interest(mask, vertices)
    
    cv2.polylines(image, VERTICES,False,thickness=1,color=(255, 255, 255), lineType=cv2.LINE_AA)

    
    Hough_Lines=hough_lines(Masked_Edges, 1, np.pi/180, 12, 10, 50)
    #Hough_Lines = hough_lines(img, rho, theta, threshold, min_line_len=12, max_line_gap)

    super_imposed=weighted_img(Hough_Lines, image, α=0.8, β=1, γ=0.)


    return(super_imposed)


In [26]:
def Init_Param(fifo_length=10):
    
        # parameters for the field of interest
    global right_average_SLOPE,left_average_SLOPE, right_average_INTERCEPT,left_average_INTERCEPT
    
    right_average_SLOPE=AveragedBuffer(fifo_length)
    left_average_SLOPE=AveragedBuffer(fifo_length)
    right_average_INTERCEPT=AveragedBuffer(fifo_length)
    left_average_INTERCEPT=AveragedBuffer(fifo_length)
    
        # parameters for polygon of interest 
    global YTOP,YBOT,BL,TL,VERTICES
    
    YTOP=340 #(mask top line : y ), will also be the top end of the extrapolated line 
    YBOT=540 #(mask bottom line : y),will also be the bottom end of the extrapolated line
    BL=(40,YBOT) # bottom left 150
    TL=(380,YTOP) # top left 430
    TR=(580,YTOP) # top right 510
    BR=(920,YBOT) # bottom right 900
    VERTICES= np.array([[BL,TL,TR,BR]], dtype=np.int32)
    
        # parameteres for color filters
    global lower_white,higher_white,lower_yellow,higher_yellow
    
    lower_yellow= HSL_color_picker(40,50,30)
    higher_yellow=HSL_color_picker(70,100,83) 
    
    lower_white=HSL_color_picker(0,0,75)
    higher_white=HSL_color_picker(360,45,100)
     

In [27]:
#&&&&&&&&&&&&&&
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
#white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
#clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
clip2 = VideoFileClip("test_videos/solidYellowLeft.mp4").subclip(0,5)
Init_Param(10)
yellow_clip = clip2.fl_image(process_image)#.subclip(0,5) #NOTE: this function expects color images!!
#yellow_clip = clip2.fl_image(process_image) #NOTE: this function expects color images!!
%time yellow_clip.write_videofile(yellow_output, audio=False)
#white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
#%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


 99%|█████████▉| 125/126 [00:10<00:00,  9.97it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 5.14 s, sys: 154 ms, total: 5.29 s
Wall time: 12.2 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [14]:
from IPython.display import HTML
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))
#""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [19]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidWhiteRight.mp4')
Init_Param(10)
white_clip = clip2.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:09<00:00, 23.08it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 8.28 s, sys: 900 ms, total: 9.18 s
Wall time: 10 s


In [16]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [20]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
Init_Param(100)
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|██████████| 251/251 [00:14<00:00, 17.74it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

CPU times: user 11.7 s, sys: 1.31 s, total: 13 s
Wall time: 14.7 s


In [18]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))